In [22]:
import numpy as np
import matplotlib as plt
import seaborn as sns
import pandas as pd
import datetime

# Group the participations data into sessions

### Load the participations data

In [17]:
participation = pd.read_pickle('../Data/participation_data')

In [71]:
participation.head()

,authenticated,duration,extra,index_,magnitude,origin,profile,profile_origin,profile_referrer,profile_utm_campaign,...,profile_utm_source,profile_utm_term,project,repetitions,type,when,where,session_id,session_duration,total_hits
63447,False,0,,63448,1,Unspecified,000073c3675ea9a1d0fe0ee3ca57e2bf,,,,...,,,413.0,1,Clicked to join the project,2016-07-10 17:57:04,NaN,1,0,0
57187,False,0,,57188,1,Unspecified,000073c3675ea9a1d0fe0ee3ca57e2bf,,,,...,,,413.0,1,Marked themselves a participant,2016-12-13 12:25:21,NaN,2,0,0
96053,False,0,,96054,1,Unspecified,0002b85e757486c6d80ed6f73f465eaa,NaN,NaN,NaN,...,NaN,NaN,16864.0,1,Added to dashboard,2017-02-13 08:25:39,NaN,3,0,0
588921,False,0,,1126171,1,Unspecified,0003a41bbdb3371df4c1829913f17537,NaN,NaN,NaN,...,NaN,NaN,19794.0,1,Clicked to join the project,2018-04-23 17:11:44,None,4,0,0
54638,False,0,,54639,1,Unspecified,000436aaa487e461e6e16e02ab3e89eb,NaN,NaN,NaN,...,NaN,NaN,659.0,1,Clicked to join the project,2016-12-11 11:20:42,NaN,5,0,0


In [72]:
len(participation) == len(participation['index_'].unique())

True

### Group participation HITs into sessions

In [19]:
# Convert the timestamp to datetime
participation['when'] = pd.to_datetime(participation['when'])

In [20]:
# Sort the dataset by profile_id and then when
participation = participation.sort_values(['profile', 'when'])

In [23]:
# Return the indices where two adjacent HITs have timestamps that are more than 30 minutes apart
gt_30min = participation.when.diff() > pd.Timedelta(minutes=30)

In [27]:
# Return the indices where two adjacent HITs have different user_ids
diff_profile = participation.profile != participation.profile.shift()

In [28]:
# Any indice that appears in the above arrays signifies the end of a session
session_id = (diff_profile | gt_30min).cumsum()

In [29]:
# Mark each HIT with a session_id
participation['session_id'] = session_id

### Count the number of sessions

In [30]:
sessions = participation['session_id'].unique()

In [31]:
len(sessions)

153502

### Explore some of the sessions data

In [58]:
session_counts = participation[['session_id', 'profile']].groupby(['session_id', 'profile']).size().reset_index(name='count')

In [59]:
session_counts.head()

,session_id,profile,count
0,1,000073c3675ea9a1d0fe0ee3ca57e2bf,1
1,2,000073c3675ea9a1d0fe0ee3ca57e2bf,1
2,3,0002b85e757486c6d80ed6f73f465eaa,1
3,4,0003a41bbdb3371df4c1829913f17537,1
4,5,000436aaa487e461e6e16e02ab3e89eb,1


In [61]:
session_counts.sort_values('count', ascending=False).head()

,session_id,profile,count
122523,122524,cdf3442c67095952427f82c2975fc88d,5840
46993,46994,4b4c399a82839ccb56b9a960770c5d3f,2471
55439,55440,5aab8db0481044218e02eb042e95dd8d,1754
47034,47035,4b4c399a82839ccb56b9a960770c5d3f,1648
86360,86361,8f52d4c68da61cc0dab830ed40fd970e,1572


In [62]:
session_counts.describe()

,session_id,count
count,153502.000000,153502.000000
mean,76751.500000,9.607354
std,44312.354848,48.010980
min,1.000000,1.000000
25%,38376.250000,1.000000
50%,76751.500000,1.000000
75%,115126.750000,2.000000
max,153502.000000,5840.000000


In [65]:
# Find the number of sessions that have over 6 participations in them
len(session_counts[session_counts['count'] > 6])

20251

### Add new columns to each HIT

In [67]:
# Add a column for session duration so far to the HITs
session_duration = participation['when'] - participation.groupby('session_id')['when'].transform('first')
participation['session_duration'] = session_duration.dt.seconds // 60

In [69]:
# Add a column for the total number of HITs in the 
total_hits = participation.groupby(["session_id"]).cumcount()
participation['total_hits'] = total_hits

In [89]:
# Mark the last event of the session
last_HIT = participation.groupby('session_id')['index_'].transform('last')
participation['end_of_session'] = last_HIT
participation['end_of_session'] = participation['index_'] != participation['end_of_session']

In [95]:
len(participation[participation['end_of_session'] == False])

153502